In [126]:
import numpy as np
import copy
from scipy.linalg import eigh, expm
np.set_printoptions(precision=3, suppress=True)

In [127]:
# pauli matrices
pauli = np.array([np.array([[1,0],[0,1]]), np.array([[0,1],[1,0]]), np.array([[0,-1.j],[1.j,0]]), np.array([[1,0],[0,-1]])])
pauli_tensor = np.array([[np.kron(pauli[i], pauli[j]) for i in range(4)] for j in range(4)])

# building operators 
def kronecker_pad(matrix, num_qubits, starting_site): 
    ''' pads a 1- or 2- local operator with identities on other sites to get 2^n by 2^n matrix '''
    kron_list = [np.eye(2) for i in range(num_qubits)]    
    kron_list[starting_site] = matrix
    if matrix.shape[0] == 4: 
        del kron_list[starting_site+1]
    
    padded_matrix = kron_list[0]
    for i in range(1, len(kron_list)):
        padded_matrix = np.kron(kron_list[i], padded_matrix)    
    return padded_matrix

# models
def heisenberg(num_qubits, bias_coeff=1.0, x_hopping_coeff=1.0, y_hopping_coeff=1.0, z_hopping_coeff=1.0): 
    terms = []
    for i in range(num_qubits): 
        bias = bias_coeff*kronecker_pad(pauli[3], num_qubits, i)
        terms.append(bias)
        
    for i in range(num_qubits-1): 
        z_hop = z_hopping_coeff*kronecker_pad(pauli_tensor[(3,3)], num_qubits, i)
        terms.append(z_hop)
        y_hop = y_hopping_coeff*kronecker_pad(pauli_tensor[(2,2)], num_qubits, i)
        terms.append(y_hop)
        x_hop = x_hopping_coeff*kronecker_pad(pauli_tensor[(1,1)], num_qubits, i)
        terms.append(x_hop)
    
    return sum(terms)

# used for initial guesses
def basis_state(num_qubits, i): 
    state = np.zeros(2**num_qubits)
    state[i] = 1.0 
    return state

In [139]:
def correction_state(ham, energy, state, tau=0.0001):
    #op = expm(-tau*(ham-energy))
    #correction_state = op @ state
    correction_state = ham @ state - energy*state
    return correction_state / np.linalg.norm(correction_state)

def eff_ham(ham, basis_set): 
    eff_H = np.eye(len(basis_set), dtype=complex)
    for i in range(len(basis_set)): 
        for j in range(len(basis_set)): 
            eff_H[i][j] = basis_set[i].conj().T @ ham @ basis_set[j]
    return eff_H    

def eff_overlap(basis_set): 
    eff_S = np.eye(len(basis_set), dtype=complex)
    for i in range(len(basis_set)): 
        for j in range(len(basis_set)): 
            eff_S[i][j] = basis_set[i].conj().T @ basis_set[j]
    return eff_S
    
def qdavidson_iter(ham, basis_set, tol=0.5):
    num_basis = len(basis_set)
    eff_H = eff_ham(ham, basis_set)
    eff_S = eff_overlap(basis_set)        
    evals, evecs = eigh(eff_H, eff_S)
    estates = [np.array(sum([evecs[:,i][j] * basis_set[j] for j in range(num_basis)])) for i in range(num_basis)]
        
    new_basis_set = copy.deepcopy(basis_set)
    residue_vals = []
    for i in range(num_basis): 
        val = np.linalg.norm((ham @ estates[i]) - (evals[i] * estates[i]))
        residue_vals.append(val)
        if val > tol: 
            state = correction_state(ham, evals[i], estates[i])            
            if linear_independence(state, new_basis_set, eff_S, tol): 
                '''
                eff_S = np.pad(eff_S, ((0, 1), (0, 1)), mode='constant')
                for i in range(len(new_basis_set)):
                    overlap = state.conj().T @ new_basis_set[i]
                    eff_S[i][len(new_basis_set)] = overlap
                    eff_S[len(new_basis_set)][i] = overlap
                '''
                new_basis_set.append(state)
                eff_S = eff_overlap(new_basis_set)
            
    return evals, estates, residue_vals, new_basis_set

def linear_independence(correction_vec, basis_set, eff_S, tol=0.01): 
    b = np.array([correction_vec.conj().T @ basis_set[i] for i in range(len(basis_set))])
    return np.linalg.norm(np.linalg.pinv(eff_S) @ b) < tol    

def qdavidson(ham, initial_basis_set, num_iter, tol=0.5): 
    basis_set = copy.deepcopy(initial_basis_set)
    for i in range(num_iter): 
        evals, estates, residue_vals, basis_set = qdavidson_iter(ham, basis_set, tol)
    return evals, estates, residue_vals, basis_set 

In [140]:
num_qubits = 4
ham = heisenberg(num_qubits)
evals, evecs = np.linalg.eigh(ham)
evals

array([-6.464, -5.828, -3.828, -3.   , -1.828, -1.   , -1.   , -0.172,
        0.464,  1.   ,  1.   ,  1.828,  3.   ,  3.828,  5.   ,  7.   ])

In [162]:
basis_set = [basis_state(num_qubits, i) for i in range(6)]
evals, estates, residue_vals, basis_set = qdavidson(ham, basis_set, 4, tol=0.0001)
evals

array([-6.464, -3.828, -1.828,  0.464,  1.   ,  1.828,  3.   ,  3.828,
        5.   ,  7.   ])

In [163]:
estates

[array([ 0.   +0.j,  0.   +0.j,  0.   +0.j,  0.149+0.j,  0.   +0.j,
        -0.558+0.j,  0.408+0.j,  0.   +0.j,  0.   +0.j,  0.408+0.j,
        -0.558+0.j,  0.   +0.j,  0.149+0.j,  0.   +0.j,  0.   +0.j,
         0.   +0.j]),
 array([ 0.   +0.j, -0.   +0.j,  0.   +0.j, -0.271+0.j,  0.   +0.j,
         0.653+0.j, -0.   +0.j,  0.   +0.j, -0.   +0.j, -0.   +0.j,
        -0.653+0.j,  0.   +0.j,  0.271+0.j,  0.   +0.j,  0.   +0.j,
         0.   +0.j]),
 array([ 0.   +0.j,  0.271+0.j, -0.653+0.j, -0.   +0.j,  0.653+0.j,
        -0.   +0.j,  0.   +0.j,  0.   +0.j, -0.271+0.j,  0.   +0.j,
        -0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,
         0.   +0.j]),
 array([ 0.   +0.j,  0.   +0.j, -0.   +0.j, -0.558+0.j,  0.   +0.j,
         0.149+0.j,  0.408+0.j,  0.   +0.j, -0.   +0.j,  0.408+0.j,
         0.149+0.j,  0.   +0.j, -0.558+0.j,  0.   +0.j,  0.   +0.j,
         0.   +0.j]),
 array([ 0. +0.j,  0.5+0.j, -0.5+0.j,  0. +0.j, -0.5+0.j,  0. +0.j,
        -0. +0.j,  0. +0.j, 

In [164]:
residue_vals

[2.676075093342767e-15,
 4.0248529826355284e-15,
 1.0648885158809838e-15,
 1.2381600202446532e-15,
 7.065416064076988e-16,
 4.871083751574254e-16,
 3.0847422370805075e-15,
 4.440892098500626e-16,
 1.538370149106851e-15,
 0.0]

In [150]:
basis_set

[array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([ 0.   +0.j,  0.   +0.j,  0.   +0.j, -0.   +0.j,  0.   +0.j,
         0.   +0.j,  0.707+0.j,  0.   +0.j,  0.   +0.j,  0.707+0.j,
         0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,
         0.   +0.j]),
 array([ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j]),
 array([ 0.+0.j,  0.+0.j,  0.+0.j, -0.+0.j, -0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j, -0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  